# Mais sobre `SELECT`

- temporary tables
- views
- variables
- subqueries


In [3]:
from functools import partial
from dotenv import load_dotenv
import insperautograder.jupyter as ia
import mysql.connector
import os

load_dotenv(override=True)

connection = mysql.connector.connect(
    host=os.getenv("MD_DB_SERVER"),
    user=os.getenv("MD_DB_USERNAME"),
    password=os.getenv("MD_DB_PASSWORD"),
    port=int(os.getenv("MD_DB_PORT", 3306)),
    database="sakila"
)

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print("Executando query:")
        for result in cursor.execute(query, multi=True):
            if result.with_rows:
                for row in result.fetchall():
                    print(row)
            else:
                print(f"{result.rowcount} linhas afetadas.")

db = partial(run_db_query, connection)

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (10061)

## Exercícios para entrega

Esta aula tem atividade para entrega, confira os prazos e exercícios

In [4]:
ia.tasks()

|    | Atividade    | De                  | Até                 | Conta como ATV?   | % Nota Atraso   |
|---:|:-------------|:--------------------|:--------------------|:------------------|:----------------|
|  0 | newborn      | 2025-08-11 00:00:00 | 2025-11-30 00:00:00 | Não               | 0%              |
|  1 | select01     | 2025-08-14 00:00:00 | 2025-08-22 23:59:59 | Sim               | 25%             |
|  2 | ddl          | 2025-08-25 12:00:00 | 2025-08-31 23:59:59 | Sim               | 25%             |
|  3 | dml          | 2025-08-28 16:30:00 | 2025-09-04 23:59:59 | Sim               | 25%             |
|  4 | agg_join     | 2025-09-01 14:00:00 | 2025-09-07 23:59:59 | Sim               | 25%             |
|  5 | group_having | 2025-09-04 16:30:00 | 2025-09-11 23:59:59 | Sim               | 25%             |
|  6 | views        | 2025-09-08 07:30:00 | 2025-09-14 23:59:59 | Sim               | 25%             |

In [5]:
ia.grades(by="task")

|    | Tarefa       |   Nota | Conta como ATV?   |
|---:|:-------------|-------:|:------------------|
|  0 | newborn      |     10 | Não               |
|  1 | select01     |     10 | Sim               |
|  2 | ddl          |     10 | Sim               |
|  3 | dml          |     10 | Sim               |
|  4 | agg_join     |     10 | Sim               |
|  5 | group_having |     10 | Sim               |
|  6 | views        |      0 | Sim               |

In [6]:
ia.grades(task="views")

|    | Atividade   | Exercício   |   Peso |   Nota |   Nota Sem Atraso |   Nota Com Atraso |
|---:|:------------|:------------|-------:|-------:|------------------:|------------------:|
|  0 | views       | ex01        |      1 |      0 |                 0 |                 0 |
|  1 | views       | ex02        |      1 |      0 |                 0 |                 0 |
|  2 | views       | ex03        |      1 |      0 |                 0 |                 0 |
|  3 | views       | ex04        |      1 |      0 |                 0 |                 0 |
|  4 | views       | ex05        |      1 |      0 |                 0 |                 0 |
|  5 | views       | ex06        |      1 |      0 |                 0 |                 0 |
|  6 | views       | ex07        |      1 |      0 |                 0 |                 0 |
|  7 | views       | ex08        |      1 |      0 |                 0 |                 0 |
|  8 | views       | ex09        |      1 |      0 |                 0 |                 0 |
|  9 | views       | ex10        |      1 |      0 |                 0 |                 0 |
| 10 | views       | ex11        |      1 |      0 |                 0 |                 0 |
| 11 | views       | ex12        |      1 |      0 |                 0 |                 0 |

In [7]:
# Média de ATV, dividindo por n-2
ia.average(excluded_count=2)

|    |   Média de ATV |
|---:|---------------:|
|  0 |             10 |

## Aquecimento

Quanta receita foi gerada para cada categoria de filmes? Mostre o nome da categoria e a receita. Ordene da maior receita para para a menor.

In [15]:
sql_ex01 = """
    SELECT c.name,
        SUM(p.amount) AS revenue
    FROM payment p
    JOIN rental r ON p.rental_id = r.rental_id
    JOIN inventory i ON r.inventory_id = i.inventory_id
    JOIN film f ON i.film_id = f.film_id
    JOIN film_category fc ON f.film_id = fc.film_id
    JOIN category c ON fc.category_id = c.category_id
    GROUP BY c.name
    ORDER BY revenue DESC;
"""

# db(sql_ex01)

In [16]:
ia.sender(answer="sql_ex01", task="views", question="ex01", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex01', style=ButtonStyle()), Output()), _dom_classes=('widget…

Liste os 10 filmes mais alugados e o numero de vezes em que foram alugados. Ordene do mais alugado para o menos alugado.

Caso exista empate, como segundo critério, ordene de forma crescente pelo nome do filme.

In [17]:
sql_ex02 = """
    SELECT f.title,
        COUNT(r.rental_id) AS rental_count
    FROM rental r
    JOIN inventory i ON r.inventory_id = i.inventory_id
    JOIN film f ON i.film_id = f.film_id
    GROUP BY f.film_id, f.title
    ORDER BY rental_count DESC, f.title ASC
    LIMIT 10;
"""

# db(sql_ex02)

In [18]:
ia.sender(answer="sql_ex02", task="views", question="ex02", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex02', style=ButtonStyle()), Output()), _dom_classes=('widget…

**DESAFIO:** Liste os filmes e o numero de vezes em que foram alugados apenas para filmes que foram alugados mais do que a média de numero de alugueis por filme. *Dica*: Serão necessários múltiplos SELECTs. Faça cada um em uma célula diferente.

In [ ]:
db("""
    SELECT
    f.film_id,
    f.title,
    COUNT(r.rental_id) AS rental_count
    FROM film f
    JOIN inventory i ON i.film_id = f.film_id
    LEFT JOIN rental r ON r.inventory_id = i.inventory_id
    GROUP BY f.film_id, f.title;
""")
db("""
    SELECT
    AVG(rental_count) AS avg_rentals_per_film
    FROM (
    SELECT
        f.film_id,
        COUNT(r.rental_id) AS rental_count
    FROM film f
    JOIN inventory i      ON i.film_id = f.film_id
    LEFT JOIN rental r    ON r.inventory_id = i.inventory_id
    GROUP BY f.film_id
    ) t;
""")
db("""
    SELECT
    t.title,
    t.rental_count
    FROM (
    SELECT
        f.film_id,
        f.title,
        COUNT(r.rental_id) AS rental_count
    FROM film f
    JOIN inventory i      ON i.film_id = f.film_id
    LEFT JOIN rental r    ON r.inventory_id = i.inventory_id
    GROUP BY f.film_id, f.title
    ) t
    WHERE t.rental_count > (
    SELECT
        AVG(rental_count)
    FROM (
        SELECT
        f.film_id,
        COUNT(r.rental_id) AS rental_count
        FROM film f
        JOIN inventory i      ON i.film_id = f.film_id
        LEFT JOIN rental r    ON r.inventory_id = i.inventory_id
        GROUP BY f.film_id
    ) x
    )
    ORDER BY t.rental_count DESC, t.title ASC;
""")
db("""
-- SUA QUERY AQUI!
""")

## Views

Uma *view* é uma tabela virtual, construida a partir de um comando `SELECT`. Por exemplo: execute o código a seguir.

In [ ]:
db("""
DROP VIEW IF EXISTS movie_count;
""")

db("""
CREATE VIEW movie_count AS
    SELECT 
        title, COUNT(rental_id) as cnt
    FROM
        film
        LEFT OUTER JOIN inventory USING (film_id)
        LEFT OUTER JOIN rental USING (inventory_id)
    GROUP BY
        film_id
    ORDER BY
        cnt ASC;
""")

Agora temos uma *view* chamada `movie_count`. Vamos verificar que ela funcionou, listando as 30 primeiras linhas:

In [ ]:
db("""
SELECT 
    *
FROM
    movie_count 
LIMIT 
    30
""")

In [ ]:
db("""
SELECT 
    * 
FROM 
    movie_count 
ORDER BY
    cnt DESC
LIMIT 30
""")

Agora suponha que alteramos a tabela `film`, mudando o nome do filme "DAZED PUNK" para "STONED PUNK".

**Atividade**: Do it.

In [ ]:
db("""
    UPDATE film
    SET title = 'STONED PUNK'
    WHERE title = 'DAZED PUNK';
""")

Verifique agora a nossa *view*:

In [ ]:
db("""
SELECT
    *
FROM
    movie_count
LIMIT
    30
""")

Como você pode ver, as views são tabelas virtuais que são **automaticamente atualizadas quando as tabelas originais são modificadas**.

Sempre que você realizar modificações nos dados, dê `commit` ou `rollback`. Ainda, evite executar múltiplas vezes as linhas de código que criem a conexão sem antes ter fechado a conexão ativa.

Vamos desfazer as alterações:

In [ ]:
connection.rollback()

Conferindo:

In [ ]:
db("""
SELECT
    *
FROM
    movie_count
LIMIT
    30
""")

### Vamos praticar

Verifique quantas vezes o filme "COWBOY DOOM" foi alugado usando a view `movie_count`

In [21]:
sql_ex03 = """
    SELECT cnt
    FROM movie_count
    WHERE title = 'COWBOY DOOM';
"""

# db(sql_ex03)

In [22]:
ia.sender(answer="sql_ex03", task="views", question="ex03", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex03', style=ButtonStyle()), Output()), _dom_classes=('widget…

Registre, na loja id=1, um aluguel do filme `"COWBOY DOOM"` (com o menor id possível), feito pelo funcionario "Jon Stephens" para o cliente "JESSIE BANKS", na data '2019-01-01', com data de retorno '2019-01-08'.

**Dicas**:

- Utilize `DESCRIBE` para descobrir se o `rental_id` precisa ser informado ou se é autoincremento (se for autoincremento, ele já receberá automaticamente o menor id possível)ç
- Será necessário utilizar `SELECT` com `LIMIT 1` para buscar os **ids** do funcionário e inventário do filme.

In [23]:
sql_ex04 = """
    INSERT INTO rental (rental_date, inventory_id, customer_id, return_date, staff_id)
    SELECT
    '2019-01-01',                                                                 -- rental_date (use DATETIME se preferir: '2019-01-01 00:00:00')
    (
        SELECT i.inventory_id
        FROM inventory i
        JOIN film f USING (film_id)
        WHERE UPPER(f.title) = 'COWBOY DOOM'  -- filme
        AND i.store_id = 1                  -- loja
        ORDER BY i.inventory_id ASC           -- “menor id possível”
        LIMIT 1
    ) AS inventory_id,
    (
        SELECT c.customer_id
        FROM customer c
        WHERE UPPER(c.first_name) = 'JESSIE'
        AND UPPER(c.last_name)  = 'BANKS'
        ORDER BY c.customer_id
        LIMIT 1
    ) AS customer_id,
    '2019-01-08',                                                                 -- return_date (idem sobre DATETIME)
    (
        SELECT s.staff_id
        FROM staff s
        WHERE UPPER(s.first_name) = 'JON'
        AND UPPER(s.last_name)  = 'STEPHENS'
        ORDER BY s.staff_id
        LIMIT 1
    ) AS staff_id;
"""

# db(sql_ex04)

In [ ]:
db("""
-- SUA QUERY AQUI CASO NECESSITE DE MAIS!
-- Crie quantas forem necesárias!
""")

In [24]:
ia.sender(answer="sql_ex04", task="views", question="ex04", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex04', style=ButtonStyle()), Output()), _dom_classes=('widget…

Verifique usando a view que a contagem de alugueis do filme subiu.

In [ ]:
db("""
-- SUA QUERY AQUI!
""")

Faça o *rollback* desta alteração, para não modificar a nossa querida base de dados *sakila*.

In [ ]:
connection.rollback()

## Tabelas temporárias

Tabelas temporárias podem ser criadas para ajudar nas tarefas de manipulação de dados. Essas tabelas existem apenas pela duração da sessão. Para criar uma tabela temporária, basta adicionar a palavra-chave `TEMPORARY` no momento da criação.

É comum criar tabelas temporárias à partir do resultado de comandos `SELECT`. Por exemplo, a seguinte query permite montar uma tabela temporária com os filmes que duram mais que 3 horas:

In [ ]:
db("""
DROP TABLE IF EXISTS long_film
""")

db("""
CREATE TEMPORARY TABLE long_film 
    SELECT
        *
    FROM
        film
    WHERE
        film.length > 180;
""")

Podemos verificar que a tabela `long_film` agora existe:

In [ ]:
db("DESCRIBE long_film")

In [ ]:
db("CALL sys.table_exists('sakila', 'long_film', @table_type);")
db("SELECT @table_type;")

Muito embora ela não apareça na lista de tabelas: isso é um bug do MySQL. (https://dev.mysql.com/worklog/task/?id=648)

In [ ]:
db("SHOW TABLES")

Vamos listar o conteudo desta tabela:

In [ ]:
db("SELECT title FROM long_film")

Vamos apagar a tabela `long_film`:

In [ ]:
db("DROP TABLE long_film")

### Vamos praticar

- Crie uma tabela temporária `max_duration` que contém a duração máxima de filme para cada categoria. Apresente o id da categoria, seu nome e a duração máxima.

**Obs**: Como nome das colunas, utilize (nesta ordem): `category_id`, `name`, `max_len`.

In [25]:
# Executamos o DROP apenas localmente, sem enviar ao servidor
# db("DROP TABLE IF EXISTS max_duration")

sql_ex05 = """
    CREATE TEMPORARY TABLE max_duration
    SELECT 
        c.category_id,
        c.name,
        MAX(f.length) AS max_len
    FROM category c
    JOIN film_category fc ON c.category_id = fc.category_id
    JOIN film f          ON fc.film_id = f.film_id
    GROUP BY c.category_id, c.name;
"""

# db(sql_ex05)

In [26]:
ia.sender(answer="sql_ex05", task="views", question="ex05", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex05', style=ButtonStyle()), Output()), _dom_classes=('widget…

 - Verifique a tabela.

In [ ]:
db("SELECT * FROM max_duration")

In [ ]:
db("DESCRIBE max_duration")

- Agora use a tabela temporária para construir uma consulta com as categorias e seus respectivos filmes mais longos:

In [ ]:
db("""
SELECT
    fc.category_id,
    md.name,
    f.film_id,
    f.title,
    f.length
FROM
    film f
    INNER JOIN film_category fc USING (film_id)
    INNER JOIN max_duration md USING (category_id)
WHERE
    f.length = md.max_len
""")

Executando query:
(1, 'Action', 212, 'DARN FORRESTER', 185)
(1, 'Action', 991, 'WORST BANGER', 185)
(2, 'Animation', 349, 'GANGS PRIDE', 185)
(2, 'Animation', 690, 'POND SEATTLE', 185)
(3, 'Children', 344, 'FURY MURDER', 178)
(3, 'Children', 993, 'WRONG BEHAVIOR', 178)
(4, 'Classics', 180, 'CONSPIRACY SPIRIT', 184)
(5, 'Comedy', 182, 'CONTROL ANTHEM', 185)
(6, 'Documentary', 973, 'WIFE TURN', 183)
(6, 'Documentary', 996, 'YOUNG LANGUAGE', 183)
(7, 'Drama', 473, 'JACKET FRISCO', 181)
(8, 'Family', 499, 'KING EVOLUTION', 184)
(9, 'Foreign', 198, 'CRYSTAL BREAKING', 184)
(9, 'Foreign', 821, 'SORORITY QUEEN', 184)
(10, 'Games', 141, 'CHICAGO NORTH', 185)
(11, 'Horror', 24, 'ANALYZE HOOSIERS', 181)
(11, 'Horror', 535, 'LOVE SUICIDES', 181)
(12, 'Music', 426, 'HOME PITY', 185)
(13, 'New', 340, 'FRONTIER CABIN', 183)
(14, 'Sci-Fi', 817, 'SOLDIERS EVOLUTION', 185)
(15, 'Sports', 813, 'SMOOCHY CONTROL', 184)
(16, 'Travel', 609, 'MUSCLE BRIGHT', 185)
(16, 'Travel', 872, 'SWEET BROTHERHOOD', 185)

- delete a tabela temporária

In [ ]:
db("""
DROP TABLE max_duration
""")

## Variáveis

Podemos montar uma query que retorne um valor só e armazenar este valor em uma variável, para uso posterior em outras queries. Para isso vamos usar o prefixo '@' para indicar variáveis, e o comando `SELECT ... INTO`.

Exemplo: quais são os filmes "caros" da nossa base sakila? Vamos descobrir quais filmes custam mais que um desvio padrão acima da média de preços de locação.

Primeiro vamos calcular a média e o desvio padrão dos preços de aluguel:

In [ ]:
db("""
SELECT 
    AVG(rental_rate), 
    STDDEV(rental_rate)
INTO 
    @avg_rate, 
    @stddev_rate 
FROM
    film;
""")

Note que a query não retorna um resultado: o resultado foi armazenado direto nas variáveis `@avg_rate` e `@stddev_rate`. Vamos usar um `SELECT` sem tabelas para ver o resultado:

In [ ]:
db("SELECT @avg_rate, @stddev_rate")

Agora podemos selecionar os filmes caros!

In [ ]:
db("""
SELECT 
    title, rental_rate
FROM
    film
WHERE
    rental_rate > @avg_rate + @stddev_rate
LIMIT 10
""")

### Vamos praticar

Armazene na variável temporária `max_films` a quantidade de filmes feitos pelo ator ou atriz que mais participou de filmes.

In [27]:
sql_ex06 = """
    SELECT MAX(film_count) INTO @max_films
    FROM (
        SELECT 
            fa.actor_id,
            COUNT(fa.film_id) AS film_count
        FROM film_actor fa
        GROUP BY fa.actor_id
    ) t;
"""

# db(sql_ex06)

In [28]:
db("""
SELECT @max_films
""")

NameError: name 'db' is not defined

In [29]:
ia.sender(answer="sql_ex06", task="views", question="ex06", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex06', style=ButtonStyle()), Output()), _dom_classes=('widget…

## Operador `IN`

Suponha que desejamos listar todos os filmes dos 3 atores mais populares. Podemos começar listando os 3 atores mais populares:

In [ ]:
db("""
SELECT 
    actor_id, first_name, last_name, COUNT(film_id) AS num_films
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
GROUP BY 
    actor_id
ORDER BY 
    num_films DESC
LIMIT 3
""")

Vamos criar uma tabela temporária para guardar a informação de `actor_id` desses atores:

In [ ]:
db("DROP TABLE IF EXISTS temp_pop_actors")
db("""
CREATE TEMPORARY TABLE temp_pop_actors
    SELECT first_name, last_name, actor_id FROM
        actor
        INNER JOIN film_actor USING (actor_id)
    GROUP BY 
        actor_id
    ORDER BY 
        COUNT(film_id) DESC
    LIMIT 3
""")
db("""
SELECT * from temp_pop_actors
""")

Por fim, vamos usar essa informação para listar os filmes dos atores populares:

In [ ]:
db("""
SELECT DISTINCT
    title
FROM
    film
    INNER JOIN film_actor USING (film_id)
WHERE
    actor_id IN (SELECT actor_id FROM temp_pop_actors);
""")

Note o uso de *subqueries*!

Não se esqueça de limpar tudo no final!

In [ ]:
db("DROP TABLE temp_pop_actors")

### Vamos praticar

Liste os atores (id, nome e sobrenome) que participaram dos 3 filmes mais rentáveis (aqueles que mais geraram receita para a locadora) ordenados pelo id do ator de modo crescente. Para isso, crie uma tabela temporária contendo o id do filme e a quantia total e use essa tabela para listar os atores.

In [ ]:
db("""
-- SUA QUERY CRIA a tabela temporaria AQUI!
""")
db("""
-- SUA QUERY que UTILIZA a tabela temporaria AQUI!
""")

In [33]:
sql_ex07 = ["""
    CREATE TEMPORARY TABLE film_revenue AS
    SELECT 
        i.film_id,
        SUM(p.amount) AS total_amount
    FROM payment p
    JOIN rental   r ON p.rental_id   = r.rental_id
    JOIN inventory i ON r.inventory_id = i.inventory_id
    GROUP BY i.film_id
    ORDER BY total_amount DESC
    LIMIT 3;
""",
"""
    SELECT DISTINCT
        a.actor_id,
        a.first_name,
        a.last_name
    FROM actor a
    JOIN film_actor fa ON fa.actor_id = a.actor_id
    WHERE fa.film_id IN (SELECT film_id FROM film_revenue)
    ORDER BY a.actor_id ASC;
"""]

ia.sender(answer="sql_ex07", task="views", question="ex07", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex07', style=ButtonStyle()), Output()), _dom_classes=('widget…

## Subqueries

Os tópicos discutidos acima poderiam ser resolvidos, em grande parte, com subqueries. As subqueries são queries `SELECT` criadas dentro de outras queries. 

Poderíamos ter usado subqueries nos mesmos lugares onde usamos tabelas temporárias, nos tópicos acima. Quando a subquery pode ser transformada em uma tabela temporária independente, separada da query exterior, dizemos que a subquery é **não-correlacionada** com a query exterior.

Usar subqueries não-correlacionadas é um tópico controverso: podemos sempre usar uma tabela temporária ou, ás vezes, pensar em um `JOIN` simples. Aliás, muitas vezes o otimizador de queries do banco de dados transformará a subquery em `JOIN`, se isso for vantajoso em termos de desempenho.

Uma subquery que depende da query externa (e portanto não pode ser separada em uma tabela temporária independente) é chamada de **subquery correlacionada**. Nestes casos podemos ter que executar a subquery para cada linha da query exterior! 

### Exemplo 1 (subquery)

Query para retornar o autor e a quantidade de músicas vinculadas a cada autor.

```sql
USE musica;
```

- Versão **sem subquery**:

```sql
SELECT 
    a.Nome_Autor, COUNT(*) qtde_musicas
FROM
    AUTOR a
        INNER JOIN
    MUSICA_AUTOR ma USING (Codigo_Autor)
GROUP BY Codigo_Autor;
```

- Versão **com subquery**:

```sql
SELECT 
    a.Nome_Autor,
    (SELECT 
            COUNT(*)
        FROM
            MUSICA_AUTOR ma
        WHERE
            ma.Codigo_Autor = a.Codigo_Autor) qtde_musicas
FROM
    AUTOR a;
```

### Exemplo 2 (subquery)

Query para retornar o autor e a média de duração de suas músicas, apenas para as músicas que possuem `a` em seu título e, então, filtrar apenas os autores com duração média entre `3.5` e `3.8`.

```sql
USE musica;
```

- Versão **sem subquery** usando `HAVING`:

```sql
SELECT 
    a.Nome_Autor AS nome, AVG(m.Duracao) AS duracao_media
FROM
    MUSICA AS m
        INNER JOIN
    MUSICA_AUTOR AS ma USING (Codigo_Musica)
        INNER JOIN
    AUTOR AS a USING (Codigo_Autor)
WHERE
    m.Nome_Musica LIKE '%a%'
GROUP BY a.Codigo_Autor
HAVING AVG(m.Duracao) BETWEEN 3.5 AND 3.8
ORDER BY duracao_media DESC;
```

- Versão **com subquery**:

```sql
SELECT 
    t.nome, t.duracao_media
FROM -- Vamos utilizar subquery
    (SELECT 
        a.Nome_Autor AS nome, AVG(m.Duracao) AS duracao_media
    FROM
        MUSICA AS m
    INNER JOIN MUSICA_AUTOR AS ma USING (Codigo_Musica)
    INNER JOIN AUTOR AS a USING (Codigo_Autor)
    WHERE
        m.Nome_Musica LIKE '%a%'
    GROUP BY a.Codigo_Autor
    ORDER BY duracao_media DESC) AS t -- Vamos criar um alias para a nossa subquery
WHERE
    t.duracao_media BETWEEN 3.5 AND 3.8;
```

### Vamos praticar

Vamos refazer a atividade dos atores dos filmes mais rentáveis, usando subqueries. 

Temos um problema: o MySQL não suporta ``LIMIT`` em subqueries com o operador ``IN``. Vamos investigar isso mais de perto. 

Em primeiro lugar faça uma tradução direta da implementação da atividade anterior trocando tabela temporária por subquery.

In [ ]:
try:
    db("""
-- SUA QUERY AQUI!
    """)
except mysql.connector.ProgrammingError as e:
    print(f"ProgrammingError: {e}")

Ok, apareceu o problema. Mas considere que o problema original não precisava de IN desde o começo! Construa essa solução.

In [34]:
sql_ex08 = """
    SELECT DISTINCT
        a.actor_id,
        a.first_name,
        a.last_name
    FROM actor a
    JOIN film_actor fa ON fa.actor_id = a.actor_id
    JOIN (
        /* Top 3 filmes por receita (subquery) */
        SELECT i.film_id
        FROM payment p
        JOIN rental   r ON r.rental_id   = p.rental_id
        JOIN inventory i ON i.inventory_id = r.inventory_id
        GROUP BY i.film_id
        ORDER BY SUM(p.amount) DESC
        LIMIT 3
    ) top3 ON top3.film_id = fa.film_id
    ORDER BY a.actor_id ASC;
"""

# db(sql_ex08)

In [35]:
ia.sender(answer="sql_ex08", task="views", question="ex08", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex08', style=ButtonStyle()), Output()), _dom_classes=('widget…

# `UNION`

Quando duas tabelas tem **EXATAMENTE** as mesmas colunas, podemos concatená-las e formar uma grande tabela unificada usando o operador `UNION`. Por exemplo: suponha que desejamos montar uma lista dos nomes e sobrenomes de todos os clientes E de todos os funcionários. Eis uma solução possível:

In [ ]:
db("DROP TABLE IF EXISTS nomes_clientes")
db("""
CREATE TEMPORARY TABLE nomes_clientes 
    SELECT first_name, last_name FROM customer
""")

In [ ]:
db("DESCRIBE nomes_clientes")
db("SELECT * FROM nomes_clientes LIMIT 5")

In [ ]:
db("DROP TABLE IF EXISTS nomes_staff")
db("""
CREATE TEMPORARY TABLE nomes_staff 
    SELECT first_name, last_name FROM staff
""")

In [ ]:
db("DESCRIBE nomes_staff")
db("SELECT * FROM nomes_staff LIMIT 5")

In [ ]:
db("DROP TABLE IF EXISTS nomes_all")
db("""
CREATE TEMPORARY TABLE nomes_all 
    (SELECT * FROM nomes_staff)
    UNION 
    (SELECT * FROM nomes_clientes)
""")

In [ ]:
db("DESCRIBE nomes_all")
db("SELECT * FROM nomes_all LIMIT 5")

In [ ]:
db("DROP TABLE IF EXISTS nomes_clientes")
db("DROP TABLE IF EXISTS nomes_staff")
db("DROP TABLE IF EXISTS nomes_all")

**Vamos praticar:** refaça o exemplo acima mas use *subqueries* ao invés de *temp tables*. Ordene de forma ascendente por first_name e last_name. Também é possível resolver apenas com união de `SELECT`s, sem *subquery*.

In [37]:
sql_ex09 = """
    SELECT first_name, last_name FROM customer
    UNION
    SELECT first_name, last_name FROM staff
    ORDER BY first_name ASC, last_name ASC
"""

# db(sql_ex09)

In [38]:
ia.sender(answer="sql_ex09", task="views", question="ex09", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex09', style=ButtonStyle()), Output()), _dom_classes=('widget…

## Desafios!

Faça uma lista de filmes que tenham mais de dois atores cujo nome inicia com a mesma letra do título do filme! Apresente o nome e a quantidade de atores. Ordene pelo id do filme. Dica: Pesquise a função `LEFT`

In [39]:
sql_ex10 = """
    SELECT 
        f.title AS film,
        COUNT(*) AS actor_count
    FROM film f
    JOIN film_actor fa ON fa.film_id = f.film_id
    JOIN actor a       ON a.actor_id = fa.actor_id
    WHERE LEFT(UPPER(a.first_name), 1) = LEFT(UPPER(f.title), 1)
    GROUP BY f.film_id, f.title
    HAVING COUNT(*) > 2
    ORDER BY f.film_id ASC;
"""

# db(sql_ex10)

In [40]:
ia.sender(answer="sql_ex10", task="views", question="ex10", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex10', style=ButtonStyle()), Output()), _dom_classes=('widget…

Semana do "DAN HARRIS": liste os clientes (nome e sobrenome) que nunca assistiram um filme do ator "DAN HARRIS" ou que já assistiram mas a ultima vez em que assistiram um filme dele foi antes de '2005-06-01'. Ordene pelo nome e sobrenome do cliente.

Considerem que a pessoa pode ter assistido um filme de Dan Harris se:

- Devolveu em 2005-06-01 ou depois
- Alugou em 2005-06-01 ou depois
- Alugou antes de 2005-06-01 e ainda não devolveu

In [43]:
sql_ex11 = """
    SELECT 
        c.first_name,
        c.last_name
    FROM customer c
    WHERE NOT EXISTS (
        SELECT 1
        FROM actor a
        JOIN film_actor fa   ON fa.actor_id   = a.actor_id
        JOIN inventory i     ON i.film_id     = fa.film_id
        JOIN rental r        ON r.inventory_id = i.inventory_id
        WHERE a.first_name = 'DAN'
            AND a.last_name  = 'HARRIS'
            AND r.customer_id = c.customer_id
            AND (
                r.rental_date >= '2005-06-01'         -- alugou em/apos 2005-06-01
            OR r.return_date >= '2005-06-01'         -- devolveu em/apos 2005-06-01
            OR (r.rental_date < '2005-06-01' AND r.return_date IS NULL)  -- alugou antes e não devolveu
            )
        )
    ORDER BY c.first_name ASC, c.last_name ASC;
"""
# db(sql_ex11)

In [44]:
ia.sender(answer="sql_ex11", task="views", question="ex11", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex11', style=ButtonStyle()), Output()), _dom_classes=('widget…

- Faça uma consulta que retorne, para cada ator, a seguinte informação:

| first_name | last_name | filmes por categoria |
|--|--|--|
| PENELOPE | GUINESS | Animation: ANACONDA CONFESSIONS; Children: LANGUAGE COWBOY; Classics: COLOR PHILADELPHIA, WESTWARD SEABISCUIT; Comedy: VERTIGO NORTHWEST; Documentary: ACADEMY DINOSAUR; Family: KING EVOLUTION, SPLASH GUMP; Foreign: MULHOLLAND BEAST; Games: BULWORTH COMMANDMENTS, HUMAN GRAFFITI; Horror: ELEPHANT TROJAN, LADY STAGE, RULES HUMAN; Music: WIZARD COLDBLOODED; New: ANGELS LIFE, OKLAHOMA JUMANJI; Sci-Fi: CHEAPER CLYDE; Sports: GLEAMING JAWBREAKER |
| NICK | WAHLBERG | Action: BULL SHAWSHANK; Animation: FIGHT JAWBREAKER; Children: JERSEY SASSY; Classics: DRACULA CRYSTAL, GILBERT PELICAN; Comedy: MALLRATS UNITED, RUSHMORE MERMAID; Documentary: ADAPTATION HOLES; Drama: WARDROBE PHANTOM; Family: APACHE DIVINE, CHISUM BEHAVIOR, INDIAN LOVE, MAGUIRE APACHE; Foreign: BABY HALL, HAPPINESS UNITED; Games: ROOF CHAMPION; Music: LUCKY FLYING; New: DESTINY SATURDAY, FLASH WARS, JEKYLL FROGMEN, MASK PEACH; Sci-Fi: CHAINSAW UPTOWN, GOODFELLAS SALUTE; Travel: LIAISONS SWEET, SMILE EARRING |
| etc | etc | etc |

Ordene pelo nome e sobrenome do ator.

Dica: use `GROUP_CONCAT` para agrupar todas as strings de uma coluna em uma string só, e `CONCAT` para unir strings particulares.

In [45]:
sql_ex12 = """
SELECT 
a.first_name,
  a.last_name,
  GROUP_CONCAT(t.cat_block ORDER BY t.category SEPARATOR '; ') AS `filmes por categoria`
FROM actor a
JOIN (
  SELECT 
    fa.actor_id,
    c.name AS category,
    CONCAT(c.name, ': ', GROUP_CONCAT(DISTINCT f.title ORDER BY f.title SEPARATOR ', ')) AS cat_block
  FROM film_actor fa
  JOIN film f           ON f.film_id = fa.film_id
  JOIN film_category fc ON fc.film_id = f.film_id
  JOIN category c       ON c.category_id = fc.category_id
  GROUP BY fa.actor_id, c.category_id, c.name
) AS t
  ON t.actor_id = a.actor_id
GROUP BY a.actor_id, a.first_name, a.last_name
ORDER BY a.first_name ASC, a.last_name ASC;
"""

# db(sql_ex12)

In [46]:
ia.sender(answer="sql_ex12", task="views", question="ex12", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex12', style=ButtonStyle()), Output()), _dom_classes=('widget…

## Conclusão

Façamos uma pausa para apreciar quão longe estamos: já conseguimos criar nossas tabelas, inserir informação, removê-la, atualizá-la, e consultar nossa base de maneiras bem sofisticadas! Vimos desde `SELECT` simples até buscas mais complexas envolvendo várias etapas de processamento para obter o dado desejado.

Por hoje é só, feche sua conexão e bom descanso!

In [ ]:
connection.close()

## Conferir Notas

Confira se as notas na atividade são as esperadas!

Primeiro na atividade atual!

In [47]:
ia.grades(by="TASK", task="views")

|    | Tarefa   |   Nota | Conta como ATV?   |
|---:|:---------|-------:|:------------------|
|  0 | views    |     10 | Sim               |

In [48]:
ia.grades(task="views")

|    | Atividade   | Exercício   |   Peso |   Nota |   Nota Sem Atraso |   Nota Com Atraso |
|---:|:------------|:------------|-------:|-------:|------------------:|------------------:|
|  0 | views       | ex01        |      1 |     10 |                10 |                 0 |
|  1 | views       | ex02        |      1 |     10 |                10 |                 0 |
|  2 | views       | ex03        |      1 |     10 |                10 |                 0 |
|  3 | views       | ex04        |      1 |     10 |                10 |                 0 |
|  4 | views       | ex05        |      1 |     10 |                10 |                 0 |
|  5 | views       | ex06        |      1 |     10 |                10 |                 0 |
|  6 | views       | ex07        |      1 |     10 |                10 |                 0 |
|  7 | views       | ex08        |      1 |     10 |                10 |                 0 |
|  8 | views       | ex09        |      1 |     10 |                10 |                 0 |
|  9 | views       | ex10        |      1 |     10 |                10 |                 0 |
| 10 | views       | ex11        |      1 |     10 |                10 |                 0 |
| 11 | views       | ex12        |      1 |     10 |                10 |                 0 |

In [49]:
ia.grades(by="task")

|    | Tarefa       |   Nota | Conta como ATV?   |
|---:|:-------------|-------:|:------------------|
|  0 | newborn      |     10 | Não               |
|  1 | select01     |     10 | Sim               |
|  2 | ddl          |     10 | Sim               |
|  3 | dml          |     10 | Sim               |
|  4 | agg_join     |     10 | Sim               |
|  5 | group_having |     10 | Sim               |
|  6 | views        |     10 | Sim               |

Média de ATV:

In [50]:
# Média de ATV, dividindo por n-2
ia.average(excluded_count=2)

|    |   Média de ATV |
|---:|---------------:|
|  0 |             10 |